In [1]:
import numpy as np
import numpy as np
import pandas as pd
from osgeo import gdal
import os
import time

In [2]:
tao = 10**-3
threshold_stop = 10**-15
threshold_step = 10**-15
threshold_residual = 10**-15
residual_memory = []

#construct a user function

def my_Func(pData,xData):
#     m1,m2,m3,m4,m5,m6=pData
    m1 = pData[0,0]
    m2 = pData[1,0]
    m3 = pData[2,0]
    m4 = pData[3,0]
    m5 = pData[4,0]
    m6 = pData[5,0]

    return m1 + m2 /(1 + np.exp(-m3 * (xData-m4))) - m2/(1 + np.exp(-m5 * (xData-m6)))    

#calculating the derive of pointed parameter,whose shape is (num_data,1)
def cal_deriv(params,input_data,param_index):
    params1 = params.copy()
    params2 = params.copy()
    params1[param_index,0] += 0.000001
    params2[param_index,0] -= 0.000001
    data_est_output1 = my_Func(params1,input_data)
    data_est_output2 = my_Func(params2,input_data)
    return (data_est_output1 - data_est_output2) / 0.000002

#calculating jacobian matrix,whose shape is (num_data,num_params)
def cal_Jacobian(params,input_data):
    num_params = np.shape(params)[0]
    num_data = np.shape(input_data)[0]
    J = np.zeros((num_data,num_params))
    for i in range(0,num_params):
        J[:,i] = list(cal_deriv(params,input_data,i))
    return J

#calculating residual, whose shape is (num_data,1)
def cal_residual(params,input_data,output_data):
    data_est_output = my_Func(params,input_data)
    residual = output_data - data_est_output
    return residual


#get the init u, using equation u=tao*max(Aii)
def get_init_u(A,tao):
    m = np.shape(A)[0]
    Aii = []
    for i in range(0,m):
        Aii.append(A[i,i])
    u = tao*max(Aii)
    return u
    
#LM algorithm
def LM(num_iter,params,input_data,output_data):
    num_params = np.shape(params)[0]#the number of params
    k = 0#set the init iter count is 0
    #calculating the init residual
    residual = cal_residual(params,input_data,output_data)
    #calculating the init Jocobian matrix
    Jacobian = cal_Jacobian(params,input_data)
    
    A = Jacobian.T.dot(Jacobian)#calculating the init A
    g = Jacobian.T.dot(residual)#calculating the init gradient g
    stop = (np.linalg.norm(g, ord=np.inf) <= threshold_stop)#set the init stop
    u = get_init_u(A,tao)#set the init u
    v = 2#set the init v=2
    
    while((not stop) and (k<num_iter)):
        k+=1
        while(1):
            Hessian_LM = A + u*np.eye(num_params)#calculating Hessian matrix in LM
            step = np.linalg.inv(Hessian_LM).dot(g)#calculating the update step
            if(np.linalg.norm(step) <= threshold_step):
                stop = True
            else:
                new_params = params + step#update params using step
                new_residual = cal_residual(new_params,input_data,output_data)#get new residual using new params
                rou = (np.linalg.norm(residual)**2 - np.linalg.norm(new_residual)**2) / (step.T.dot(u*step+g))
                if rou > 0:
                    params = new_params
                    residual = new_residual
#                     residual_memory.append(np.linalg.norm(residual)**2)
                    Jacobian = cal_Jacobian(params,input_data)#recalculating Jacobian matrix with new params
                    A = Jacobian.T.dot(Jacobian)#recalculating A
                    g = Jacobian.T.dot(residual)#recalculating gradient g
                    stop = (np.linalg.norm(g, ord=np.inf) <= threshold_stop) or (np.linalg.norm(residual)**2 <= threshold_residual)
                    u = u*max(1/3,1-(2*rou-1)**3)
                    v = 2
                else:
                    u = u*v
                    v = 2*v
            if(rou > 0 or stop):
                break;
        
    return params
 

In [3]:
class Dataset:
    def __init__(self, in_file):
        self.in_file = in_file  # Tiff或者ENVI文件

        dataset = gdal.Open(self.in_file)
        self.XSize = dataset.RasterXSize  # 网格的X轴像素数量
        self.YSize = dataset.RasterYSize  # 网格的Y轴像素数量
        self.Bands = dataset.RasterCount  # 波段数
        self.GeoTransform = dataset.GetGeoTransform()  # 投影转换信息
        self.ProjectionInfo = dataset.GetProjection()  # 投影信息
    
    def get_data(self):
        #band: 读取第几个通道的数据
        dataset = gdal.Open(self.in_file)
        data = dataset.ReadAsArray(0,0,self.XSize,self.YSize)
        return data
    

    def get_lon_lat(self):
        #获取经纬度信息
        gtf = self.GeoTransform
        x_range = range(0, self.XSize)
        y_range = range(0, self.YSize)
        x, y = np.meshgrid(x_range, y_range)
        lon = gtf[0] + x * gtf[1] + y * gtf[2]
        lat = gtf[3] + x * gtf[4] + y * gtf[5]
        
        lon_lat=[]
        for (longitude,latitude) in zip(lon,lat):
            lon_lat.append(list(zip(longitude,latitude)))
            
        return np.array(lon_lat)
    
    def new_dataset(self,data,lon_lat):
        new_dataset=[]
        for i in range(self.YSize):
            for j in range(self.XSize):
                x1 = lon_lat[i,j]
                x2 = data[:,i,j]
                x=np.hstack((x1,x2))
                new_dataset.append(x)
            
        return np.array(new_dataset)
    
    def dataset2dim(self,data):
        dataset2dim=[]
        for i in range(self.YSize):
            for j in range(self.XSize):
#                 x1 = lon_lat[i,j]
                x2 = data[:,i,j]
#                 x=np.hstack((x1,x2))
                dataset2dim.append(x2)

        return np.array(dataset2dim)

In [4]:
start=time.time()

dir_path = r"D:\Desktop\mypaper\data"
filename = "gee-EVI-10884.tif"
file_path = os.path.join(dir_path, filename)
dataset = Dataset(file_path)

data = dataset.get_data( ) 

lon_lat = dataset.get_lon_lat()  # 获取经纬度信息longitude, latitude
# new_dataset=dataset.new_dataset(data,lon_lat)
dataset2dim=dataset.dataset2dim(data)

In [5]:
dataset2dim=np.expand_dims(dataset2dim,axis=2)
xdim=dataset2dim.shape[0]
ydim=dataset2dim.shape[1]
zdim=dataset2dim.shape[2]
print(xdim,ydim,zdim)

pInput=[900,5000,0.1,140,0.1,270]
pData=np.expand_dims(np.repeat([pInput],xdim,axis=0), axis=2)
print('pData',pData.shape)
xInput=np.linspace(9, 361, 23)
xData=np.expand_dims(np.repeat([xInput],xdim,axis=0), axis=2)
print('xData',xData.shape)

item=np.repeat([1000000],xdim,axis=0)
print('item',item.shape)
print('dataset2dim',dataset2dim.shape)

12656 23 1
pData (12656, 6, 1)
xData (12656, 23, 1)
item (12656,)
dataset2dim (12656, 23, 1)


In [6]:
result=np.array(list(map(LM,item,pData,xData,dataset2dim)))
print('total time',time.time()-start)
print(result.shape)
print(result)

d:\python37\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in exp


total time 400.7306115627289
(12656, 6, 1)
[[[3.47149409e+02]
  [4.97796748e+03]
  [5.67322682e-02]
  [9.79390083e+01]
  [3.24581264e-02]
  [3.13485719e+02]]

 [[2.59123084e+02]
  [5.55123959e+03]
  [5.11233809e-02]
  [1.03009874e+02]
  [2.78772412e-02]
  [3.13219237e+02]]

 [[1.68087996e+02]
  [5.83235128e+03]
  [6.98737098e-02]
  [9.38572814e+01]
  [2.37052342e-02]
  [3.27735065e+02]]

 ...

 [[1.25022425e+03]
  [4.47185020e+03]
  [1.13159383e-01]
  [1.10513342e+02]
  [4.22808072e-02]
  [2.78764809e+02]]

 [[1.22781691e+03]
  [4.55367245e+03]
  [1.20397968e-01]
  [1.09644469e+02]
  [5.42168363e-02]
  [2.74877528e+02]]

 [[1.22781691e+03]
  [4.55367245e+03]
  [1.20397968e-01]
  [1.09644469e+02]
  [5.42168363e-02]
  [2.74877528e+02]]]
